In [1]:
import mhkit.strain as strain
import pandas as pd
import xarray as xr

## Load strain data and map rosettes to TBLR (top, bottom, left, right)
4 rosettes, 2 pairs of 2 rosettes



In [2]:
raw_data = pd.read_csv(
    "data/strain/R17.csv",
    sep=",",
    header=None,
    index_col=0,
    names=[
        "time",
        "ec_1",
        "eb_1",
        "ea_1",
        "ec_2",
        "eb_2",
        "ea_2",
        "ec_3",
        "eb_3",
        "ea_3",
        "ec_4",
        "eb_4",
        "ea_4",
    ],
)
# TODO - make time a timeseries variable
raw_data

,ec_1,eb_1,ea_1,ec_2,eb_2,ea_2,ec_3,eb_3,ea_3,ec_4,eb_4,ea_4
time,,,,,,,,,,,,
0.00,0.379,0.208,2.926,-0.623,1.160,0.270,0.309,1.877,-0.038,2.376,-0.312,-1.811
0.01,-2.578,1.360,1.523,0.192,-1.943,1.429,3.128,1.372,1.539,0.018,0.689,-1.305
0.02,0.839,1.862,2.198,-0.820,-1.347,0.785,1.124,2.687,0.594,0.370,-0.465,0.039
0.03,-5.249,-0.961,1.248,-0.881,-1.231,1.252,0.893,0.718,0.024,0.654,0.489,0.989
0.04,-3.377,1.078,2.092,-0.504,-0.504,1.268,-0.821,-0.989,0.577,-4.508,-0.312,0.514
...,...,...,...,...,...,...,...,...,...,...,...,...
1407.35,13.287,3.473,-18.279,-2.061,-0.428,5.256,25.280,11.563,25.195,-0.634,-21.175,-37.170
1407.36,16.541,5.171,-18.800,-3.036,-5.307,7.300,23.141,14.141,26.548,-2.776,-20.132,-35.298
1407.37,16.722,5.529,-15.892,-2.061,-4.517,6.495,22.500,13.679,25.744,-1.433,-19.965,-38.288


In [3]:
# Map all data to ea,eb,ec variables with time and rosette as dimensions
ea = xr.DataArray(raw_data[['ea_1', 'ea_2', 'ea_3', 'ea_4']])
ea = ea.assign_coords(dim_1=[1, 2, 3, 4])
ea = ea.rename({"dim_1": "rosette"})

eb = xr.DataArray(raw_data[['eb_1', 'eb_2', 'eb_3', 'eb_4']])
eb = eb.assign_coords(dim_1=[1, 2, 3, 4])
eb = eb.rename({"dim_1": "rosette"})

ec = xr.DataArray(raw_data[['ec_1', 'ec_2', 'ec_3', 'ec_4']])
ec = ec.assign_coords(dim_1=[1, 2, 3, 4])
ec = ec.rename({"dim_1": "rosette"})

data = xr.Dataset(data_vars={"ea": ea, "eb": eb, "ec": ec})
data

<xarray.Dataset> Size: 15MB
Dimensions:  (time: 140740, rosette: 4)
Coordinates:
  * time     (time) float64 1MB 0.0 0.01 0.02 ... 1.407e+03 1.407e+03 1.407e+03
  * rosette  (rosette) int64 32B 1 2 3 4
Data variables:
    ea       (time, rosette) float64 5MB 2.926 0.27 -0.038 ... 26.64 -38.96
    eb       (time, rosette) float64 5MB 0.208 1.16 1.877 ... 12.31 -21.14
    ec       (time, rosette) float64 5MB 0.379 -0.623 0.309 ... 24.27 -1.606

## Define system parameters


In [4]:
radius = 0  # -30 # rotation angle of the root
width = 44.6024 / 1000  # width of strain gauge cube holding the blade root [m]
height = 44.6024 / 1000  # height of strain gauge cube holding the blade root [m]
radius = 15.24 / 1000  # radius of blade root [m]
elastic_modulus = 197e9  # [Pa]
shear_modulus = 77.4e9  # [Pa]
blade_span = 0.86868  # [m]

## Define orientation in strain rosette (90deg or 120deg) and calculate ex, ey, exy
Each rosette is a 90 degree strain rosette measuring 3 strains: c (axial), b (diagonal), a (shear)

for each rosette, calculate axial, shear, coupling components (depends on 90 or 120deg rosette)

In [5]:
axial_strain_x, axial_strain_y, shear_strain = strain.calculate_primary_strains(
    data["ec"], data["eb"], data["ea"], 90
)
data = data.assign(axial_strain_x=axial_strain_x)
data = data.assign(axial_strain_y=axial_strain_y)
data = data.assign(shear_strain=shear_strain)
data

<xarray.Dataset> Size: 28MB
Dimensions:         (time: 140740, rosette: 4)
Coordinates:
  * time            (time) float64 1MB 0.0 0.01 0.02 ... 1.407e+03 1.407e+03
  * rosette         (rosette) int64 32B 1 2 3 4
Data variables:
    ea              (time, rosette) float64 5MB 2.926 0.27 ... 26.64 -38.96
    eb              (time, rosette) float64 5MB 0.208 1.16 ... 12.31 -21.14
    ec              (time, rosette) float64 5MB 0.379 -0.623 ... 24.27 -1.606
    axial_strain_x  (time, rosette) float64 5MB 0.379 -0.623 ... 24.27 -1.606
    axial_strain_y  (time, rosette) float64 5MB 2.926 0.27 ... 26.64 -38.96
    shear_strain    (time, rosette) float64 5MB -1.445 1.337 ... -13.15 -0.857

## Calculate moments and torsion from strain
### Method 1 (theoretical method):
get N, Mx (flapwise if radius=0) from et and eb
get N, My (edgewise if radius=0) from el, er
if rotated, use Mx, My to get M_flap, M_edge
each strain rosette (exy on a given side) can estimate torque
typically measure torsion at multiple places and average in some way
assuming that there's a thin wall in between blade root and measurement surface
works with any rectangular block with 4 strain rosettes

In [6]:
normal_topBottom, moment_x, torsion_top, torsion_bottom = strain.calculate_loads(
    data.sel(rosette=1),
    data.sel(rosette=2),
    elastic_modulus,
    shear_modulus,
    width,
    height,
    radius,
)
normal_rightLeft, moment_y, torsion_right, torsion_left = strain.calculate_loads(
    data.sel(rosette=3),
    data.sel(rosette=4),
    elastic_modulus,
    shear_modulus,
    height,
    width,
    radius,
)

# Calculate the average torsion
torsion = (torsion_top + torsion_bottom + torsion_right + torsion_left)/4

In [7]:
# Next we

<xarray.DataArray 'axial_strain_x' (time: 140740)> Size: 1MB
array([-3.02759979e+07, -2.96059553e+08,  2.35755721e+06, ...,
        1.81916560e+09,  1.45299974e+09,  1.63378715e+09])
Coordinates:
  * time     (time) float64 1MB 0.0 0.01 0.02 ... 1.407e+03 1.407e+03 1.407e+03

### Method 2 (more generalized):
not transforming moments from one coordinate to another, but transforming the strains
"equivalent strain" because they often half 1x or 1/2x factor
calculate look up table. Moment = blade constant * strain
torsion is simpler regression and essentially the same as Method 1
in this experiment flapwise pull was found to be best for the regression
constants are dependent on blade, root, measurements

NOTE - do not try and implement this through MHKiT functions. Could add an example of the methodology here, but not worth making functions for